

### html比对：找出不同点的文本，在原文本上添加标识和ID，并提取出来

In [1]:
from bs4 import BeautifulSoup

In [76]:
# html1_doc = open("./htmls/case5.pdf.html").read()
# html2_doc = open("./htmls/case6.pdf.html").read()
html1_doc = open("./htmls/table1.html").read()
html2_doc = open("./htmls/table2.html").read()

def extract_body(html_doc):
    """
    !!!
    """
    html_doc =  html_doc.replace("\t","")
# #     f1 = open("./case3.html","r").read()
    soup = BeautifulSoup(html_doc, "html.parser")
#     sidebar = soup.find('div', id='sidebar')
#     if sidebar:
#         sidebar.extract()
#     # [s.extract() for s in soup("head")]
#     # print(ss)
# #     tt = soup("body")[0].extract()
    return str(soup)

html1_doc = extract_body(html1_doc)
html2_doc = extract_body(html2_doc)

In [19]:
# html1_doc

In [17]:
new_html_doc = []
for i in range(len(html1_doc)):
    new_html_doc.append((html1_doc[i],i))
new_html_doc[0:10]

[('<', 0),
 ('!', 1),
 ('D', 2),
 ('O', 3),
 ('C', 4),
 ('T', 5),
 ('Y', 6),
 ('P', 7),
 ('E', 8),
 (' ', 9)]

In [32]:
html1_doc = html1_doc.replace(">\n<","><")
start_index = html1_doc.index("<body")
def keeplr(item):
    if item[0] in [">","<"] and item[1]>=start_index:
        return True
flags = [("<",0)]+list(filter(keeplr,new_html_doc))
flags[0:10]

[('<', 0),
 ('<', 3119),
 ('>', 3126),
 ('<', 3128),
 ('>', 3134),
 ('<', 3136),
 ('>', 3164),
 ('<', 3166),
 ('>', 3300),
 ('<', 3302)]

In [77]:
def extract_content_with_format(html_doc):
    html_doc = html_doc.replace(">\n<","><")
    start_index = html_doc.index("<body")
    new_html_doc = []
    for i in range(len(html_doc)):
        new_html_doc.append((html_doc[i],i))

#     flags = list(filter(keeplr,new_html_doc))
    flags = [("<",0)]+list(filter(keeplr,new_html_doc))
    newnew_html_doc = []
    contents = []
    for i in range(0,len(flags)-1):
    #     print(flags[i][0])
    #     print(flags[i+1][0])
    #     print(flags[i+1][1]-flags[i][1])
        if flags[i][0] == ">" and flags[i+1][0] == "<" and (flags[i+1][1]-flags[i][1])>1:
            contents.append(html_doc[flags[i][1]+1:flags[i+1][1]])
            newnew_html_doc.append(flags[i][0]+"%s")
        else:
            newnew_html_doc.append(html_doc[flags[i][1]:flags[i+1][1]])
    newnew_html_doc.append(html_doc[flags[-1][1]:])
    return "".join(newnew_html_doc),contents
formats1,contents1 = extract_content_with_format(html1_doc)
formats2,contents2 = extract_content_with_format(html2_doc)
# contents1

NameError: name 'keeplr' is not defined

In [ ]:
# contents2

In [ ]:
# "1212".index("2")

In [7]:
def locate_str(string,target):
    import re
    a = [m.start() for m in re.finditer(target, string)]
    return a

string=u'<body > <td> </td>'
target = u'</td>'
locate_str(string,target)

[13]

In [8]:
POS = []
def locate_tables(string):
    pos1 = locate_str(string,"<table")
    pos2 = locate_str(string,"</table>")
#     print(string[pos1[0]:pos2[0]+10])
#     print(pos1)
#     print(pos2)
    pos = []
    for i in range(len(pos1)):
        pos.append((pos1[i],pos2[i]))
    return pos
POS = locate_tables(html1_doc)
# POS

In [9]:
# html1_doc[23400:23400+100]

In [10]:
def is_in_tables(POS,pos =()):
    for p in POS:
        if p[0]<pos[0] and p[1] > pos[1]:
            return True
    return False
is_in_tables(POS,(19710, 23413))

True

In [78]:
def extract_content_with_format_td(html_doc):
    html_doc = html_doc.replace(">\n<","><").replace("\n","")
    POS = locate_tables(html_doc)
#     print(html_doc[POS[0][0]:POS[0][1]+10])
#     print(POS)
    if "<body" in html_doc:
        start_index = html_doc.index("<body")
    else:
        start_index = 0
    new_html_doc = []
    for i in range(len(html_doc)):
        new_html_doc.append((html_doc[i],i))
#     print(new_html_doc[0:20])
    def keeplr(item):
        """是否是尖括号"""
        if item[0] in [">","<"] and item[1]>=start_index:
            return True
    # flags = list(filter(keeplr,new_html_doc))
    flags = [("<", 0)] + list(filter(keeplr, new_html_doc))
    newnew_html_doc = []
    contents = []
    for i in range(0,len(flags)-1):
        if flags[i][0] == ">" and flags[i+1][0] == "<" and (flags[i+1][1]-flags[i][1])>1:
            s_cc = flags[i][1]+1
            e_cc = flags[i+1][1]
            cc = html_doc[s_cc:e_cc]
            intable_flag = is_in_tables(POS,(s_cc,e_cc))
            if len(cc)>0:
                contents.append((cc,intable_flag,(s_cc,e_cc)))
#                 print(cc)
                newnew_html_doc.append(flags[i][0]+"%s")
            else:
                newnew_html_doc.append(html_doc[flags[i][1]:flags[i + 1][1]])
        else:
            newnew_html_doc.append(html_doc[flags[i][1]:flags[i+1][1]])
    newnew_html_doc.append(html_doc[flags[-1][1]])  # ">"　last element
    # newnew_html_doc.append(">")
    return "".join(newnew_html_doc),contents
formats1,contents1 = extract_content_with_format_td(html1_doc)
formats2,contents2 = extract_content_with_format_td(html2_doc)


In [85]:
for c in contents1:
    print(c)

('a', False, (1129, 1130))
('a', False, (1149, 1150))
('b', False, (1157, 1158))
('b', False, (1177, 1178))
('c', False, (1185, 1186))
('c', False, (1205, 1206))
('121212', True, (1723, 1729))
('23', True, (3050, 3052))
('2213', True, (3928, 3932))
('Bcd', True, (4159, 4162))


In [86]:
for c in contents2:
    print(c)

('ggbbddee', False, (1148, 1156))
('121212', True, (1599, 1605))
('23', True, (2926, 2928))
('345', True, (3823, 3826))
('Abch', True, (4060, 4064))
('345', True, (4746, 4749))


### 获取每个部分的长度

In [79]:
def get_lenofline(contents):
    """
    获取每个段落的长度
    """
    lenoflines = []
    for i in range(len(contents)):
        if contents[i][1]: # table
#             print(contents[i])
            lenoflines.append(1)
        else:
            lenoflines.append(len(contents[i][0]))
    return lenoflines
right_lens = get_lenofline(contents2)
left_lens = get_lenofline(contents1)
right_lens

[8, 1, 1, 1, 1, 1]

In [80]:
left_lens

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [81]:
def sum_lens(lens):
    """
    lens:[1,2,3]
    new_lens: [1,3,6]
    """
    new_lens = []
    for i in range(len(lens)):
        new_lens.append(sum(lens[0:i+1]))
    return new_lens

left_indexes = sum_lens(left_lens)
right_indexes = sum_lens(right_lens)
left_indexes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [82]:
left_indexes[0:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
right_indexes[0:20]

[2, 6, 11, 12]

### 拉成字符串比对

In [2]:
import difflib
class DocxDiff(difflib.HtmlDiff):
    def __init__(self):
        super().__init__()
#         self.MAX_PARA_LEN = config["MAX_PARA_LEN"]
        
    def format_diff(self,fromlines, tolines, fromdesc='', todesc='',context=False, numlines=5, *, charset='utf-8'):
        self._make_prefix()

        # change tabs to spaces before it gets more difficult after we insert
        # markup
        fromlines,tolines = self._tab_newline_replace(fromlines,tolines)

        # create diffs iterator which generates side by side from/to data
        if context:
            context_lines = numlines
        else:
            context_lines = None
        diffs = difflib._mdiff(fromlines,tolines,context_lines,linejunk=self._linejunk,
                      charjunk=self._charjunk)
        
        return list(diffs)

In [ ]:
def contents2blocks(contents):
    blocks = []
    for content in contents:
        if content[1]:  # 表格
            blocks.append(content[0])
        else:
            blocks += list(content[0])
    return blocks
# contents2blocks(contents1)

In [46]:
diff_result = differ.format_diff(["345","12",""],
                                 ["345","","12"])
diff_result

[((1, '345'), (1, '345'), False),
 (('', '\n'), (2, '\x00+ \x01'), True),
 ((2, '12'), (3, '12'), False),
 ((3, '\x00- \x01'), ('', '\n'), True)]

In [6]:
# todo
differ = DocxDiff()
diff_result = differ.format_diff(["20200226","\x06","\x06","\x06","26","3"],
                                 ["20200226","\x06","27","26","0226","3"]
                                )
diff_result

[((1, '20200226'), (1, '20200226'), False),
 ((2, '\x06'), (2, '\x06'), False),
 ((3, '\x00-\x06\x01'), (3, '\x00+27\x01'), True),
 ((4, '\x00-\x06\x01'), ('', '\n'), True),
 ((5, '26'), (4, '26'), False),
 (('', '\n'), (5, '\x00+0226\x01'), True),
 ((6, '3'), (6, '3'), False)]

In [18]:
diff_result = differ.format_diff(["202002^2 6"],
                                 ["202002^1 6"]
                                )
diff_result

[((1, '202002^\x00^2\x01 6'), (1, '202002^\x00^1\x01 6'), True)]

In [17]:
diff_result = differ.format_diff(["2020","\x06","\x06","26","3"],
                                 ["2020","26","0226","\x06","3"]
                                )
diff_result

[((1, '2020'), (1, '2020'), False),
 (('', '\n'), (2, '\x00+26\x01'), True),
 (('', '\n'), (3, '\x00+0226\x01'), True),
 ((2, '\x06'), (4, '\x06'), False),
 ((3, '\x00-\x06\x01'), ('', '\n'), True),
 ((4, '\x00-26\x01'), ('', '\n'), True),
 ((5, '3'), (5, '3'), False)]

In [75]:
diff_result = differ.format_diff(["2020","26","3"],
                                 ["2020","26","0226","3"]
                                )
diff_result

[((1, '2020'), (1, '2020'), False),
 ((2, '26'), (2, '26'), False),
 (('', '\n'), (3, '\x00+0226\x01'), True),
 ((3, '3'), (4, '3'), False)]

In [72]:
diff_result = differ.format_diff("".join(["2020","26","3"]),
                                 "".join(["2020","26","0226","3"])
                                )
diff_result

[((1, '2'), (1, '2'), False),
 ((2, '0'), (2, '0'), False),
 ((3, '2'), (3, '2'), False),
 ((4, '0'), (4, '0'), False),
 ((5, '2'), (5, '2'), False),
 ((6, '6'), (6, '6'), False),
 (('', '\n'), (7, '\x00+0\x01'), True),
 (('', '\n'), (8, '\x00+2\x01'), True),
 (('', '\n'), (9, '\x00+2\x01'), True),
 (('', '\n'), (10, '\x00+6\x01'), True),
 ((7, '3'), (11, '3'), False)]

In [64]:
diff_result = differ.format_diff(["20200226","\x06","26","3","部分空格","20200226","0226","","4"],
                                 ["20200226","\x06","26","4"]
                                )
diff_result

[((1, '20200226'), (1, '20200226'), False),
 ((2, '\x06'), (2, '\x06'), False),
 ((3, '26'), (3, '26'), False),
 ((4, '\x00-3\x01'), ('', '\n'), True),
 ((5, '\x00-部分空格\x01'), ('', '\n'), True),
 ((6, '\x00-20200226\x01'), ('', '\n'), True),
 ((7, '\x00-0226\x01'), ('', '\n'), True),
 ((8, '\x00- \x01'), ('', '\n'), True),
 ((9, '4'), (4, '4'), False)]

In [57]:
diff_result = differ.format_diff(["20200226","\x06","0226","\x06","26","\x06","2","\x06","20200226","\x06","26","\x06","3"],
                                 ["20200226","\06","0226","\x06","2","\x06","20200226","\x06","26","\x06","0226","\x06","3"]
                                )
diff_result

[((1, '20200226'), (1, '20200226'), False),
 ((2, '\x06'), (2, '\x06'), False),
 ((3, '0226'), (3, '0226'), False),
 ((4, '\x00-\x06\x01'), ('', '\n'), True),
 ((5, '\x00-26\x01'), ('', '\n'), True),
 ((6, '\x06'), (4, '\x06'), False),
 ((7, '2'), (5, '2'), False),
 ((8, '\x06'), (6, '\x06'), False),
 ((9, '20200226'), (7, '20200226'), False),
 ((10, '\x06'), (8, '\x06'), False),
 ((11, '26'), (9, '26'), False),
 ((12, '\x06'), (10, '\x06'), False),
 (('', '\n'), (11, '\x00+0226\x01'), True),
 (('', '\n'), (12, '\x00+\x06\x01'), True),
 ((13, '3'), (13, '3'), False)]

In [9]:
### differ = DocxDiff()
# diff_result = differ.format_diff(["345","12","2345","\x06",
#                                   "\x06","34","\x06","\x06",
#                                   "\x06","\x06","12345","\x06"],
#                                  ["345","12","2345","\x06",
#                                   "\x06","34","345","\x06",
#                                   "\x06","\x06","345","\x06"])

diff_result = differ.format_diff(["345","12","2345","\06","34","\06","12345","\x06"],
                                 ["345","12","2345","\06","34","345","\06","345","\x06"])
diff_result

[((1, '345'), (1, '345'), False),
 ((2, '12'), (2, '12'), False),
 ((3, '2345'), (3, '2345'), False),
 ((4, '\x06'), (4, '\x06'), False),
 ((5, '34'), (5, '34'), False),
 (('', '\n'), (6, '\x00+345\x01'), True),
 ((6, '\x06'), (7, '\x06'), False),
 ((7, '\x00-12\x01345'), (8, '345'), True),
 ((8, '\x06'), (9, '\x06'), False)]

In [ ]:
def re_arrange(diffs):
    """对于表格内容"""
    for i in range(1,len(diffs)):
        
        

In [10]:
"%s|%s" % ("1","k")

'1|k'

In [35]:
import json
differ = DocxDiff()
diff_result = differ.format_diff(contents2blocks(contents1), contents2blocks(contents2)) 
diff_result

NameError: name 'contents1' is not defined

In [32]:
# html1_doc = open("./htmls/table1.html").read()
# html2_doc = open("./htmls/table2.html").read()
# differ = DocxDiff()
# diff_result = differ.format_diff(html1_doc,html2_doc)
# for d in diff_result:
#     if d[2]:
#         print(d)

### 罗列比对左右的内容

In [51]:
%store -r diff_res

In [62]:
# def convert_difftype(diffs):
#     """
#     \x04 mod
#     \x03 del
#     \x02 add
#     "0"  不变
#     """
#     new_diffs = []
#         # 格式修改
#     for item in diffs:
#         l_index = item[0][0]
#         r_index = item[1][0]
#         l = item[0][1]  
#         r = item[1][1]
#         if "\x00-" in l and "\00+" in r:
#             l = l.replace("\x00-","").replace("\x01","")  # \x04 mod
#             r = r.replace("\x00+","").replace("\x01","")  # \x04 mod
#             le = (l_index,l)
#             re = (r_index,r)
#             new_diffs.append((le,re,"\x04","\x04"))
#         elif "\x00-" in l:
#             l = l.replace("\x00-","").replace("\x01","") # \x03  del
#             le = (l_index,l)
#             re = (r_index,r)
#             new_diffs.append((le,re,"\x03","\x03"))
#         elif "\x00+" in r:
#             r = r.replace("\x00+","").replace("\x01","")  # \x02 add
#             le = (l_index,l)
#             re = (r_index,r)
#             new_diffs.append((le,re,"\x02","\x02"))
#         else:
#             le = (l_index,l)
#             re = (r_index,r)
#             new_diffs.append((le,re,"0","0"))
#     return new_diffs
# new_diffs = convert_difftype(diff_res)
# new_diffs

In [61]:
# def judge_difftype(new_diffs,left_indexes=[],right_indexes=[]):
#     newnew_diffs = []
#     pre = None
#     for item in new_diffs:
# #         print(pre)
#         if pre == "\x04" and item[3] == "\x04" and item[1][0]-1 in right_indexes:
#             newnew_diffs.append((item[0],item[1],item[2],"\x02"))
# #             item[3] = "\x02"
#             pre = "\x02"
#         elif pre == "\x04" and item[0][0] == "" and item[1][0]-1 not in right_indexes:
#             newnew_diffs.append((item[0],item[1],"\x04","\x04"))
# #             item[3] = "\x04"
# #             item[2] = "\x04"
#             pre = "\x04"
#         else:
#             newnew_diffs.append(item)
#             pre = item[3]
#     return newnew_diffs

In [63]:
# judge_difftype(new_diffs,left_indexes=[6],right_indexes=[4,6])

In [64]:
# judge_difftype(new_diffs,left_indexes=[3],right_indexes=[4,6])

In [84]:
def append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1):
#     print(c1_index)
    if c0_index in left_indexes:
        Ldiffs[-1][0].append("")
    if c1_index in right_indexes:
        Ldiffs[-1][1].append("")
    Ldiffs[-1][0][-1] += c0  # left
    Ldiffs[-1][1][-1] += c1  # right
    

def listdiff(diffs,left_indexes=[],right_indexes=[]):
    """
    嵌套List
    外层List的index对应diff_id
    字符级别diff转为line级别的，合同相同类型的char
    """
    Lchgs = []
    Ladds = []
    Ldels = []
    Ldiffs = []
    
    Ldiffs = []
    new_diffs = []
    
    # 格式修改
    for item in diffs:
        l_index = item[0][0]
        r_index = item[1][0]
        l = item[0][1]  
        r = item[1][1]
        if "\x00-" in l and "\00+" in r:
            l = l.replace("\x00-","").replace("\x01","")  # \x04 mod
            r = r.replace("\x00+","").replace("\x01","")  # \x04 mod
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x04"))
        elif "\x00-" in l:
            l = l.replace("\x00-","").replace("\x01","") # \x03  del
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x03"))
        elif "\x00+" in r:
            r = r.replace("\x00+","").replace("\x01","")  # \x02 add
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x02"))
        else:
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"0"))
        cur = 0
        
    print(new_diffs)
    cur = None  
    # 再次格式修改  chars-->lines
    for d in new_diffs:
        c2 = d[2]
        
        c0_index = d[0][0]
        c0 = d[0][1]
        c1_index = d[1][0]
        c1 = d[1][1]
#         if c2 in ["\x02","\x03","\x04"]:
        # 外层List 类型发生变化则外层List append
        if cur != c2:
            cur = c2
            if cur == "\x02":
#                 Ladds.append([c0,c1,"add"])
                Ldiffs.append([[c0],[c1],"add",len(Ldiffs)])
            elif cur == "\x03":
#                 Ldels.append([c0,c1,"del"])
                Ldiffs.append([[c0],[c1],"del",len(Ldiffs)])
            elif cur == "\x04":
#                 Lchgs.append([c0,c1,"mod"])
                Ldiffs.append([[c0],[c1],"mod",len(Ldiffs)])
            else:
                Ldiffs.append([[c0],[c1],"0",len(Ldiffs)])
        # 内层List
        else:  # cur == c2
#             if c0_index in left_indexes:
#                 pass
            
#             if c1_index in right_indexes:
#                 pass
            
            if cur == "\x02":
#                 Ladds[-1][0] += c0 
#                 Ladds[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            elif cur == "\x03":
#                 Ldels[-1][0] += c0 
#                 Ldels[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            elif cur == "\x04":
#                 Lchgs[-1][0] += c0 
#                 Lchgs[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            else:
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
    return Ldiffs

Ldiffs = listdiff(diff_result,left_indexes,right_indexes)
Ldiffs

[((1, 'a'), (1, 'g'), '\x04'), ((2, 'a'), (2, 'g'), '\x04'), ((3, 'b'), (3, 'b'), '0'), ((4, 'b'), (4, 'b'), '0'), ((5, 'c'), (5, 'd'), '\x04'), ((6, 'c'), (6, 'd'), '\x04'), (('', '\n'), (7, 'e'), '\x02'), (('', '\n'), (8, 'e'), '\x02'), ((7, '121212'), (9, '121212'), '0'), ((8, '23'), (10, '23'), '0'), ((9, '2213'), (11, '345'), '\x04'), ((10, 'Bcd'), (12, 'Abch'), '\x04'), (('', '\n'), (13, '345'), '\x02')]


[[['a', 'a'], ['gg'], 'mod', 0],
 [['b', 'b'], ['bb'], '0', 1],
 [['c', 'c'], ['dd'], 'mod', 2],
 [['\n\n'], ['e', 'e'], 'add', 3],
 [['121212', '23'], ['121212', '23'], '0', 4],
 [['2213', 'Bcd'], ['345', 'Abch'], 'mod', 5],
 [['\n'], ['345'], 'add', 6]]

In [366]:
a = [1,2,3]
a[-1:]

[3]

In [384]:
a = ((1,2),(3,4),5)
a[0][0]

1

In [87]:
# 将有变化的地方过滤出来
def isdiff(item):
    if item[2] != "0":
        return True
diff_lines = list(filter(isdiff,Ldiffs))
# len(diff_lines)

In [88]:
# len(diff_lines[1][0])  # 57
diff_lines

[[['a', 'a'], ['gg'], 'mod', 0],
 [['c', 'c'], ['dd'], 'mod', 2],
 [['\n\n'], ['e', 'e'], 'add', 3],
 [['2213', 'Bcd'], ['345', 'Abch'], 'mod', 5],
 [['\n'], ['345'], 'add', 6]]

In [106]:
add_span = '%s<span class="add-text">%s</span>%s'
change_span = '%s<span class="mod-text">%s</span>%s'
delete_span = '%s<span class="del-text">%s</span>%s'
no_span = '%s%s%s'

def ismod(item):
    if item[2] == "mod":
        return True

def isadd(item):
    if item[2] == "add":
        return True

def isdel(item):
    if item[2] == "del":
        return True

def lenlen(paras):
    return len("".join(paras))

def get_res(from_span,to_span,cur,pre,back):
    line = {"from": from_span % ("".join(pre),"".join(cur[0]),"".join(back)),
        "to":to_span % ("".join(pre),"".join(cur[1]),"".join(back))}
    return line

def topK(Ldiffs,dt="add",K=3):
    res_lines = []
    if dt == "add":
        target_lines = list(filter(isadd, Ldiffs))
    if dt == "mod":
        target_lines = list(filter(ismod, Ldiffs))
    if dt == "del":
        target_lines = list(filter(isdel, Ldiffs))
    newLdiffs = sorted(target_lines, key=lambda x: (x[2],lenlen(x[0])), reverse=True)
    for i in range(min(K,len(newLdiffs))):
        cur = newLdiffs[i]
        print(cur)
        # todo
        if cur[2]==dt:
            index = cur[3]
            if index >0 and Ldiffs[index-1][2] == "0":
                pre = Ldiffs[index-1][0]
            else:
                pre = ""
            if index < len(Ldiffs)-1 and Ldiffs[index+1][2] == "0":  # 不变项
                back = Ldiffs[index+1][0]
            else:
                back = ""
#             res_diffs.append([[pre[-10:],cur[0],back[0:10]],[pre[-10:],cur[1],back[0:10]]])
#             print(back)
            if dt == "add":
                line =get_res(no_span,add_span,cur,pre,back)
#                 line = {"from": no_span % ("".join(pre),"".join(cur[0]),"".join(back)),
#                         "to":add_span % ("".join(pre),"".join(cur[1]),"".join(back))}
            if dt == "mod":
                line =get_res(change_span,change_span,cur,pre,back)
#                 line = {"from": change_span % (pre[-10:], cur[0], back[0:10]),
#                         "to": change_span % (pre[-10:], cur[1], back[0:10])}
            if dt == "del":
                line =get_res(delete_span,no_span,cur,pre,back)
#                 line = {"from": delete_span % (pre[-10:], cur[0], back[0:10]),
#                         "to": no_span % (pre[-10:], cur[1], back[0:10])}
            res_lines.append(line)
    return res_lines
topK(Ldiffs)

[['\n\n'], ['e', 'e'], 'add', 3]
[['\n'], ['345'], 'add', 6]


[{'from': '\n\n12121223', 'to': '<span class="add-text">ee</span>12121223'},
 {'from': '\n', 'to': '<span class="add-text">345</span>'}]

In [93]:
# 统计三类diff的个数
def diff_stat(diff_lines):
    add_count = 0
    delete_count = 0
    change_count = 0
    for line in diff_lines:
        if line[2] == "mod":
            change_count +=1
        if line[2] == "add":
            add_count +=1
        if line[2] == "del":
            delete_count +=1
#     stat = {"add": add_count, "delete": delete_count, "change": change_count}
    return {"add": add_count, "delete": delete_count, "change": change_count}

diff_stat(diff_lines)

{'add': 2, 'delete': 0, 'change': 3}

In [65]:
def left_spearte(item):
    l = item[0][1]
    r = item[1][1]
    if "\x00-" in l and "\00+" in r:
        l = l.replace("\x00-","\x04").replace("\x01","\x04")  # \x04 mod
    elif "\x00-" in l:
        l = l.replace("\x00-","\x03").replace("\x01","\x03") # \x03  del
    elif "\x00+" in l:
        l = l.replace("\x00+","\x02").replace("\x01","\x02")  # \x02 add
    if "\x05" in l:
        l = "\x05"
    return (item[0][0]-1,l)

def right_spearte(item):
    l = item[0][1]
    r = item[1][1]
    if "\x00-" in l and "\00+" in r:
        r = r.replace("\x00+","\x04").replace("\x01","\x04")
    elif "\x00-" in r:
        r = r.replace("\x00-","\x03").replace("\x01","\x03")
    elif "\x00+" in r:
        r = r.replace("\x00+","\x02").replace("\x01","\x02")
    if "\x05" in r:
        r = "\x05"
    return (item[1][0]-1,r)

def left_isok(item):
#     return True
    return item[0][0] != ""
def right_isok(item):
#     return True
    return item[1][0] != ""

def processdiffchars(diff_chars):
    """
    1. 连续
    2. left没有add
    3. right没有delete
    """
    cursor = 0
    left_res= []
    right_res = []
    left_tmp = ""
    right_tmp = ""
    while(cursor<len(diff_chars) and (not left_isok(diff_chars[cursor]) or not right_isok(diff_chars[cursor]))):
#         print(cursor)
        if left_isok(diff_chars[cursor]):
            left_res.append(left_spearte(diff_chars[cursor]))
        else:
            left_tmp = "\x02\x02\x02\x02"
        if right_isok(diff_chars[cursor]):
            right_res.append(right_spearte(diff_chars[cursor]))
        else:
            right_tmp = "\x03\x03\x03\x03"
        cursor += 1
    if cursor < len(diff_chars):
        left_res.append(left_spearte(diff_chars[cursor]))
        right_res.append(right_spearte(diff_chars[cursor]))
        left_res[-1] = (left_res[-1][0], left_tmp+ left_res[-1][1]) 
        right_res[-1] = (right_res[-1][0], right_tmp +right_res[-1][1]) 
        for i in range(cursor+1,len(diff_chars)):
            if left_isok(diff_chars[i]):
                left_res.append(left_spearte(diff_chars[i]))
            else:
                if not left_res[-1][1].endswith("\x02\x02\x02\x02"):
                    left_res[-1] = (left_res[-1][0],left_res[-1][1] + "\x02\x02\x02\x02") 

            if right_isok(diff_chars[i]):
                right_res.append(right_spearte(diff_chars[i]))
            else:
                if not right_res[-1][1].endswith("\x03\x03\x03\x03"):
                    right_res[-1] = (right_res[-1][0], right_res[-1][1]+ "\x03\x03\x03\x03" )
    return left_res,right_res
left_res,right_res = processdiffchars(diff_result)

In [66]:
# left_res

In [67]:
from functools import reduce
def mergeline(item0,item1):
    left = item0[1]
    right = item1[1]
#     print(left[-1])
#     print(right[0])
#     for 
    if len(right)>1 and len(left)>1:
#         if left[-1] == right[0] and left[-2]!="\x05" and right[1]!="\x05":
        if left[-1] == right[0]:
            return (0,left[0:-1]+right[1:])
    return (0,left+right)


# def process_table(formats,lines):
#     for i in lines

diff_prefix = "%s@%s" # left-diff_id@sub_id 


def merge_addid(diffs,lenoflines,K=3,dire="left"):
    """
    diffs:
    lens: 
    return lines
    """
    chgs = """<span class=\""""+ dire +"""-mod-text" id=%s>"""
    adds = """<span class=\""""+ dire +"""-add-text" id=%s>"""
    dels = """<span class=\""""+ dire +"""-del-text" id=%s>"""
    ends = """</span>"""
    start = 0
    diffid = 0
    res_lines = []
    mid_lines = []
    cur_diff = 0
    diffsubid = 0
    for index in range(len(lenoflines)):
        """
        先按格式分，再按diff分
        相同类型相连则算一个diff_id
        同一个diff_id下， html标签不同，则sub_diff_id不同
        """
        chars = []
        cur_line_diffs = diffs[start:start+lenoflines[index]]
        if len(cur_line_diffs)>0:
            cur_line = reduce(mergeline, cur_line_diffs)[1]
            cur_line = cur_line.replace("\x02\x02","\x02").replace("\x03\x03","\x03").replace("\x04\x04","\x04")
            mid_lines.append(cur_line)
            f_cs = []
            cur = 0
#             print(cur_line)
            if len(cur_line)>=1:
                c = cur_line[0]
                if c in ["\x02","\x03","\x04"]:
                    if cur_diff == 0:
                        diffid += 1
                        diffsubid = 0
                    else:
                        if c != cur_diff:
                            diffid += 1
                            diffsubid = 0
                        else:
                            diffsubid += 1
                    cur = c
                    cur_diff = c
                    ID = str(diffid)+"@"+str(diffsubid)
                    c = c.replace("\x02",adds % ID).replace("\x03",dels% ID).replace("\x04",chgs% ID)
                else:
                    cur_diff = 0
    #             print(f_cs) 
                f_cs.append(c)
                for i in range(1,len(cur_line)):
                    c = cur_line[i]
                    if c in ["\x02","\x03","\x04"]:
    #                     print(c)
                        if c != cur:
                            diffid += 1
                            cur = c
                            cur_diff = c
                            diffsubid = 0
                            ID = str(diffid)+"@"+str(diffsubid)
                            c = c.replace("\x02",adds % ID).replace("\x03",dels% ID).replace("\x04",chgs% ID)
                        else:
                            cur = 0
                            c = ends
#                     else:
#                         cur_diff = 0
                    f_cs.append(c)
                res_line = "".join(f_cs)
                if len(cur_line)>1:
#                     pass
#                     print(cur_line[-1])
                    if cur_line[-1] not in ["\x02","\x03","\x04"]:
                        cur_diff = 0
            else:
                res_line = ""
        else:
            res_line = ""
        res_lines.append(res_line)
        start += lenoflines[index]
    return res_lines,mid_lines

#     for i in range(1,len(diffs)):
# res_lines,mid_lines = merge_addid(right_res,right_lens)
res_lines,mid_lines = merge_addid(left_res,left_lens)
# res_lines

In [68]:
right_res_lines,right_mid_lines = merge_addid(right_res,right_lens)

In [69]:
def add_formats(formats,new_lines):
    fs = formats.split("%s")
    finals = []
    for i in range(len(fs) - 1):
        finals.append(fs[i])
        finals.append(new_lines[i])
    finals.append(fs[-1])
    #     return formats % tuple(new_lines)
    return "".join(finals)
# res = add_formats(formats2,res_lines)
res = add_formats(formats1,res_lines)

In [70]:
right_res = add_formats(formats2,right_res_lines)

In [484]:
from bs4 import BeautifulSoup
import re
def extract_spans(html_doc):

    soup = BeautifulSoup(html_doc, "html.parser")
#     soup.find_all(class_=re.compile("itl"))
#     text = soup.find_all('span',class_="left-del-text")
    text = soup.find_all('span',class_=re.compile("left-"))
#     print(text)
    return text
#     if sidebar:
#         sidebar.extract()
#     return str(soup)

spans = extract_spans(res)
# for sp in spans:
#     print(sp)
#     if "left" in sp:
#         print(sp)

In [71]:
open("./table_res.html","w").write(res)

5398

In [72]:
open("./table_right.html","w").write(right_res)

5327

In [255]:
a /= 2

TypeError: unsupported operand type(s) for /=: 'list' and 'int'